# Lambda
## I. 基本概念
- lambda: an expression form that generates function objects.
- 和def定义的function的区别：
  - 所属类型不同：
    - def是statement，lambda是expression，因此lambda可以用在def不能用的地方。
      - 比如：lambda可以直接做为list中的元素，也可以作为argument传参。但是def不能直接用，只能用def定义好函数后的函数名。
    - 也因为lambda的整个定义必须在一条expression中写完，功能有限，所以只用来表达简单的函数。
  - 工作方式不同：
    - def的工作方式是在namespace中新建一个name，同时新建一个function object，再将name refer到该function object上。之后可以通过<code>func_name()</code>的方式call function。
    - lambda也会新建一个function object，但是不会新建name。如果把lambda新建的funciton对象赋值给一个name，那么也可以通过该name来call function
    - <font color=green>lambda通常也是这样用的，但它的优点是整个函数以expression的形式定义在一行中，很方便</font>
- 和def相似的地方：
  - arguments的scope一样
  - 工作方式也一样

## II. 典型使用场景
#### 1. nested function

In [1]:
def action(x):
    return lambda y: x + y
act = action(100)
act(2)

102

#### 2. 结合条件判断

In [2]:
lower = lambda x, y: x if x < y else y
lower(2, 3)

2

#### 3. 结合loops

In [3]:
s = ['hello', 'world', '!']
showall = lambda x: [print(i, end=' ') for i in x]
showall(s)

# 特例-1
M = lambda x: [x**n for n in range(2, 5)]
M(2)

hello world ! 

[4, 8, 16]

In [4]:
# 特例-2：注意两种形式之间的赋值差异
M = [lambda x: x**n for n in range(2, 5)]
for f in M:
    print(f(2), end=' ')

16 16 16 

#### 4. 作为loop类型的函数参数
- map函数用lambda和iterable为参数，返回iterable
  - <code>map(lambda x: expr(x), iterable)</code>
- filter函数用lambda和iterable为参数，返回iterable
  - <code>filter(lambda x: expr(x), iterable)</code>返回iterable
- reduce函数用lambda和iterable为参数，执行遍历计算
  - <code>reduce(lambda x: expr(x), iterable)</code>

In [16]:
# map
for i in map(lambda x: x**2, range(-5, 5)):
    print(i, end=' ')
print('\n')

# filter
for i in filter(lambda x: x > 0, range(-5, 5)):
    print(i, end=' ')
print('\n')

# reduce
from functools import reduce
r = reduce(lambda x, y: x + y, range(-5, 5))
print(r)

25 16 9 4 1 0 1 4 9 16 

1 2 3 4 

-5


#### 5. 构造jump tables
- 功能：list or dictionary of actions to be performed on demand

In [5]:
L = [lambda x: x ** 2, 
     lambda x: x ** 3, 
     lambda x: x ** 4]
for f in L:
    print(f(2), end=' ')
print('\n')

4 8 16 



## III. late binding problem
- <font color=blue>**late binding**</font>：python为了支持dynamic typing而采用了late binding的变量赋值策略。也就是resolution of function, method calls and property accesses at runtime而不是compile time。也就是说，函数在运行时才会invoke，因此相应的变量才会根据当时的条件确定names并赋值。
- <font color=blue>**binding time problem**</font>：如果函数定义在一个loop里面，并且refer了loop variable, loop遍历过程中函数没有被invoke，等到loop结束之后，生成的函数再被invoke。这时候做function resolution取到的是loop variable迭代到最后时刻的状态值。
- <font color=orange>lambda函数在loop iteration时容易出现binding time problem。</font>

In [20]:
# late binding problem
functions = []
for n in range(3):
    functions.append(lambda x: n * x)
    
results = [function(2) for function in functions]
print(results)

[4, 4, 4]


In [18]:
# 解决方式: 
# 直接把每次迭代时x的取值通过default value赋值，此时参数会先取走值，不用等函数call之后
functions = []
for n in range(3):
    functions.append(lambda x, n=n: n * x)

results = [function(2) for function in functions]
print(results)

[0, 2, 4]


In [23]:
functions = []
for n in range(3):
    functions.append(lambda x: n * x)

# n是for loop的enclosing scope中的变量
# 这里的scope层次很像nested function的结构
# 但是并没有closure生成
print(functions[0].__closure__)
print(functions[0].__code__.co_freevars)

None
()
